In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [ ]:
from grakel.graph import Graph

In [ ]:
def job_to_graph(s, task_num):
    temp = ""
    for i in s:
        if i != 'J' and i != 'M' and i != 'R':
            temp += i
        else:
            temp += '#'+i
    task = temp.split('#')[1:]
    if len(task) != task_num:
        return {}, {}
    st = set(i for i in range(1, task_num+1))
    task = sorted(task, key=lambda x:int(x[1:x.find('_') if x.find('_') != -1 else len(x)]))
    edges = {}
    node_labels = {}
    try:
        for t in task:
            temp = t.split('_')
            key = int(temp[0][1:])
            value = list(map(int, temp[1:]))
            for i in value:
                if i not in st:
                    return {}, {}
                if i not in edges:
                    edges[i] = []
                edges[i].append(key)
            if key not in st:
                return {}, {}
            node_labels[key] = temp[0][0]
        if len(node_labels) != task_num:
            return {}, {}
    except:
        return {}, {}
        
    return edges, node_labels
        


In [ ]:
file = pd.read_csv("/data/zkp/taskDAG/job0.csv")
file = file[file.odps_task_type.isin(["SQL"])]
file.columns

In [ ]:
for i in range(1, 10):
    read_file = pd.read_csv("/data/zkp/taskDAG/job"+str(i)+".csv")
    file = pd.concat([file, read_file[read_file.odps_task_type.isin(["SQL"])]])
    

In [ ]:
file[(file.fuxi_task_name=="J8_2_7M1M3M6M7_5_6R10_9R11_10R2_1R4_3R5_4R9_8")].T
# file[file.fuxi_task_name.str.len()==45].fuxi_task_name.unique()

In [ ]:
# ins = pd.read_csv("/data/zkp/instance1hour.csv")
# ins.project_name.value_counts().head(10)
# ins.project_name.value_counts().head(50)
ins[ins.project_name=="aliyun_searchlog"].fuxi_job_name.unique().shape

In [ ]:
# G = ins[["project_name", "fuxi_job_name", "fuxi_task_name", "fuxi_avg_cpu", "fuxi_avg_mem", "total_time_seconds", "DAG"]].groupby(["project_name", "DAG", "fuxi_job_name", "fuxi_task_name"]).mean()

G.query('project_name=="admin_task_project"')

In [ ]:
global_attributes = file[["project_name", "start_time", "fuxi_job_sn", "fuxi_quota_id", "fuxi_quota_name", 
                          "fuxi_inst_num", "running_mode", "plan_cpu_cost", "plan_mem_cost", "fuxi_cpu_cost"]]

In [ ]:
global_attributes = global_attributes.values
global_attributes.shape

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for i in [0, 4, 6]:
    global_attributes[:, i] = le.fit_transform(global_attributes[:, i])

In [ ]:
global_attributes = global_attributes.astype(np.float64)
global_attributes[np.isnan(global_attributes)] = 0

In [ ]:
task = file["fuxi_task_name"].values
task_num = file["fuxi_task_num"].values
count = {}
X_data, y_data = [], []
edges_data = []
node_data = []
for i in range(task.shape[0]):
    if task[i].find('_') != -1:
        edges, node_labels = job_to_graph(task[i], task_num[i])
        if len(edges):
#             print(edges, node_labels)
            X_data.append(Graph(edges, node_labels=node_labels))
            y_data.append(global_attributes[i, :])
            edges_data.append(edges)
            node_data.append(node_labels)
            if task_num[i] not in count:
                count[task_num[i]] = 0
            count[task_num[i]] += 1
y_data = np.asarray(y_data)
# print(count)

In [ ]:
print(len(X_data), y_data.shape)

In [ ]:
from grakel.kernels import WeisfeilerLehman, VertexHistogram
wl_kernel = WeisfeilerLehman(n_iter=5, normalize=False, base_graph_kernel=VertexHistogram)

In [ ]:
X_wl = wl_kernel.fit_transform(X_data)

In [ ]:
s_test = ["M1", "M1R2_1", "M2R1_2", "M1M2J3_1_2", "M2M1R3_1_2", "M1M2R3_1R4_2", "M1M2R3_2R4_1"]
num_test = [1, 2, 2, 3, 3, 4, 4]
X_test = []
for i in range(len(s_test)):
    edges, node_labels = job_to_graph(s_test[i], num_test[i])
    X_test.append(Graph(edges, node_labels=node_labels))
wl_kernel = WeisfeilerLehman(n_iter=10, normalize=True, base_graph_kernel=VertexHistogram)
wl_kernel.fit_transform(X_test)

In [ ]:
X_wl.shape

In [ ]:
class_id = {}
class_wl_kernel = {}
class_cnt = 0
for i in range(X_wl.shape[0]):
    is_new_class = 1
    belong_class = -1
    for key in class_wl_kernel:
        if X_wl[i][i] != class_wl_kernel[key]:
            continue
        for index in range(len(class_id[key])):
            if X_wl[i][class_id[key][index]] != class_wl_kernel[key]:
                break
            elif index == len(class_id[key])-1:
                is_new_class = 0
                belong_class = key
        if belong_class != -1:
            break
    if is_new_class:
        class_wl_kernel[class_cnt] = X_wl[i][i]
        class_id[class_cnt] = []
        belong_class = class_cnt
        class_cnt += 1
    class_id[belong_class].append(i)    
    


In [ ]:
class_cnt

In [ ]:
for key in class_id:
    for i in range(min(100, len(class_id[key]))):
        print(edges_data[class_id[key][i]], node_data[class_id[key][i]], end=',')
    print('\n')

In [ ]:
temp = []
for key in class_id:
    temp.append(len(class_id[key]))
print(sorted(temp))
print(sum(temp))


In [ ]:
class_label = [0 for i in range(X_wl.shape[0])]
for key in class_id:
    for i in class_id[key]:
        class_label[i] = key
X_final = np.hstack((np.asarray(class_label).reshape(-1, 1), y_data[:, :-1]))
X_final.shape

In [ ]:
temp = file[file.fuxi_task_name=="M1R2_1"]
cpu = temp.fuxi_cpu_cost.values
mem = temp.fuxi_mem_cost.values
_ = plt.hist(cpu, range=(0, 20000), bins=50)

In [ ]:
for i in range(100):
    _ = plt.hist(y_data[X_final[:, 0]==i][:, 9]/ y_data[X_final[:, 0]==i][:, 5], range=(0, 20000), bins=50)
    plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_data, test_size=0.2, random_state=42)

In [ ]:
y_train_target = y_train[:, -1]
y_test_target = y_test[:, -1]

In [ ]:
from sklearn.ensemble import RandomForestRegressor
# model = RandomForestRegressor()
model = RandomForestRegressor()
model.fit(X_train, y_train_target)
# y_pred = model.predict(K_test)

In [ ]:
print(model.score(X_train, y_train_target))
print(model.score(X_test, y_test_target))

In [ ]:
y_predict = model.predict(X_test)

In [ ]:
error = {}
for i in range(y_test_target.shape[0]):
#     print(len(class_id[X_test[i, 0]]))
#     if len(class_id[X_test[i, 0]]) < 2:
#         continue
    key = int(X_test[i, 0])
    if key not in error:
        error[key] = []
    if y_test_target[i] == 0:
        error[key].append(0)
    else:
        error[key].append(abs(y_test_target[i]-y_predict[i]) / y_test_target[i])
for key in error:
    temp = np.mean(error[key])
    if temp < 0.05 and len(class_id[key]) > 1:
        print(len(class_id[key]), temp, edges_data[class_id[key][0]])
    

In [ ]:
new_error = []
for key in error:
    new_error.append(np.mean(error[key]))
error = sorted(new_error)
error_y = 100 * np.arange(len(error)) / (len(error)-1)
plt.figure(figsize=(15, 8))
plt.plot(error, error_y)
plt.hlines(80, 0, 1)
plt.xlim(0, 1)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(25, 8))
plt.plot(range(200), y_predict[:200])
plt.plot(range(200), y_test_target[:200])
plt.yscale("log")

In [ ]:
error = []
for i in range(y_test_target.shape[0]):
    if y_test_target[i] == 0:
        error.append(0)
    else:
        error.append(abs(y_test_target[i]-y_predict[i]) / y_test_target[i])
error = sorted(error)
error_y = 100 * np.arange(len(error)) / (len(error)-1)
plt.figure(figsize=(15, 8))
plt.plot(error, error_y)
plt.xlim(0, 1)

In [ ]:
error = []
for i in range(y_test.shape[0]):
    if y_test[i] == 0:
        error.append(0)
    else:
        error.append(abs(y_test[i]-y_predict[i]) / y_test[i])
error = sorted(error)
error_y = 100 * np.arange(len(error)) / (len(error)-1)


In [ ]:
new_X_train.shape

In [ ]:
new_model = RandomForestRegressor()
new_model.fit(new_X_train, y_train)
print(new_model.score(new_X_train, y_train))
print(new_model.score(new_X_test, y_test))
new_y_predict = new_model.predict(new_X_test)

In [ ]:
plt.figure(figsize=(25, 8))
plt.plot(range(200), new_y_predict[:200])
plt.plot(range(200), y_test[:200])
plt.yscale("log")
plt.show()
new_error = []
for i in range(y_test.shape[0]):
    if y_test[i] == 0:
        new_error.append(0)
    else:
        new_error.append(abs(y_test[i]-new_y_predict[i]) / y_test[i])
new_error = sorted(new_error)
new_error_y = 100 * np.arange(len(new_error)) / (len(new_error)-1)
plt.figure(figsize=(15, 8))
plt.plot(error, error_y)
plt.plot(new_error, new_error_y)
plt.xlim(0, 1)
plt.legend(["error", "new_error"])

In [ ]:
X_train = K_train
print(X_train.shape)
X_test = K_test[:, [0, 2, 6, 8, 9]]
print(X_test.shape)
model = RandomForestRegressor()
model.fit(X_train, y_train_target)
print(model.score(X_train, y_train_target))
print(model.score(X_test, y_test_target))
y_predict = model.predict(X_test)
error = {}
for i in range(y_test_target.shape[0]):
    if knn_label[i] not in error:
        error[knn_label[i]] = []
    if y_test_target[i] == 0:
        error[knn_label[i]].append(0)
    else:
        error[knn_label[i]].append(abs(y_test_target[i]-y_predict[i]) / y_test_target[i])
new_error = []
for key in error:
    new_error.append(np.mean(error[key]))
    
error = sorted(new_error)
error_y = 100 * np.arange(len(error)) / (len(error)-1)
plt.figure(figsize=(15, 8))
plt.plot(error, error_y)
plt.xlim(0, 1)

In [ ]:
instance = pd.read_csv("/data/zkp/buy2/test/d0.csv")

In [ ]:
instance.columns

In [ ]:
Algo = instance[instance.odps_task_type=="Algo"]
for i in Algo.odps_task_name.unique():
    temp = Algo[Algo.odps_task_name==i]
    for j in temp.fuxi_task_name.unique():
        print(i, j)
        t = temp[temp.fuxi_task_name==j]
        fig, axes = plt.subplots(1, 2, figsize=(16, 6))
        for p in t.project_name.unique():
            axes[0].scatter(t[t.project_name==p].fuxi_avg_cpu.values, t[t.project_name==p].total_time_seconds.values, label=p)
#         axes[0].set_xlabel("CPU")
#         axes[0].set_ylabel("time")
        axes[0].legend()
#         for p in t.project_name.unique():
#             axes[1].scatter(t[t.project_name==p].fuxi_avg_mem.values, t[t.project_name==p].total_time_seconds.values, label=p)
#         axes[1].legend()
#         axes[1].set_xlabel("MEM")
#         axes[1].set_ylabel("time")
        plt.show()

In [ ]:
for j in Algo.fuxi_task_name.unique():
    print(j)
    t = Algo[Algo.fuxi_task_name==j]
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    for p in t.project_name.unique():
        axes[0].scatter(t[t.project_name==p].fuxi_avg_cpu.values, t[t.project_name==p].total_time_seconds.values, label=p)
    axes[0].legend()
    for p in t.project_name.unique():
        axes[1].scatter(t[t.project_name==p].fuxi_avg_mem.values, t[t.project_name==p].total_time_seconds.values, label=p)
    axes[1].legend()
    plt.show()

In [ ]:
for j in Algo.fuxi_task_name.unique():
    print(j)
    t = Algo[Algo.fuxi_task_name==j]
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    for p in t.project_name.unique():
        axes[0].scatter(t[t.project_name==p].fuxi_max_cpu.values / t[t.project_name==p].fuxi_avg_cpu.values, t[t.project_name==p].fuxi_max_mem.values / t[t.project_name==p].fuxi_avg_mem.values, label=p)
    axes[0].legend()
    plt.show()

In [ ]:
for t in instance.odps_task_type.unique():
    temp = instance[(instance.odps_task_type==t) & (~np.isnan(instance.fuxi_avg_cpu))]
    v = temp.fuxi_avg_cpu.values
    p99 = np.percentile(v, 99)
    y = plt.hist(v, range(0, int(p99)))[0]
    print(t, end=' ')
    print(sum(y) / temp.shape[0])
    plt.show()

In [ ]:
for t in instance.odps_task_type.unique():
    temp = instance[(instance.odps_task_type==t) & (~np.isnan(instance.total_time_seconds))]
    v = temp.total_time_seconds.values
    p99 = np.percentile(v, 99)
    y = plt.hist(v, range(0, 200))[0]
    print(t, end=' ')
#     print(sum(y) / temp.shape[0])
    plt.show()

In [ ]:
for t in instance.odps_task_type.unique():
    temp = instance[(instance.odps_task_type==t) & (~np.isnan(instance.fuxi_avg_mem))]
    v = temp.fuxi_avg_mem.values
    p99 = np.percentile(v, 99)
    y = plt.hist(v, range(0, int(p99)))[0]
    print(t, end=' ')
#     print(sum(y) / temp.shape[0])
    plt.show()

In [ ]:
for t in instance.odps_task_type.unique():
    temp = instance[(instance.odps_task_type==t) & (~np.isnan(instance.fuxi_avg_mem)) & (~np.isnan(instance.fuxi_max_mem)) & (instance.fuxi_avg_mem!=0)
                   & (~np.isnan(instance.fuxi_avg_cpu)) & (~np.isnan(instance.fuxi_max_cpu)) & (instance.fuxi_avg_cpu!=0)]
    mem = temp.fuxi_max_mem.values / temp.fuxi_avg_mem.values
#     p99 = np.percentile(v, 99)
#     y = plt.hist(v, range(1, 10))[0]
    print(t, end=' ')
#     print(sum(y) / temp.shape[0])

    cpu = temp.fuxi_max_cpu.values / temp.fuxi_avg_cpu.values
    plt.scatter(cpu, mem)
#     y = plt.hist(mem, range=(1, 2))[0]

    plt.xlabel("CPU Max/Avg")
    plt.ylabel("MEM Max/Avg")
    plt.show()
#     y = plt.hist(cpu)[0]
#     plt.show()

In [ ]:
SQL = instance[instance.odps_task_type=="SQL"]

In [ ]:
SQLRT = instance[instance.odps_task_type=="SQLRT"]

In [ ]:
Algo = SQLRT
M, R, J = [], [], []
MM, RR, JJ = [], [], []
MT, RT, JT = [], [], []
for j in Algo.fuxi_task_name.unique():
    if j in ["M1", "R2_1", "R3_2"]:
        continue
    t = Algo[Algo.fuxi_task_name==j]
    if j[0] == 'M':
        M.append(np.mean(t.fuxi_avg_cpu))
        MM.append(np.mean(t.fuxi_max_cpu))
        MT.append(np.mean(t.total_time_seconds))
    elif j[0] == 'R':
        R.append(np.mean(t.fuxi_avg_cpu))
        RR.append(np.mean(t.fuxi_max_cpu))
        RT.append(np.mean(t.total_time_seconds))
    elif j[0] == 'J':
        J.append(np.mean(t.fuxi_avg_cpu))
        JJ.append(np.mean(t.fuxi_max_cpu))
        JT.append(np.mean(t.total_time_seconds))
        


In [ ]:
plt.scatter(M, MM, label="M")
plt.scatter(J, JJ, label="J")
plt.scatter(R, RR, label="R")
plt.legend()
plt.xlabel("Avg")
plt.ylabel("Max")
plt.show()

In [ ]:
LOT = instance[instance.odps_task_type=="LOT"]

In [ ]:
for j in LOT.fuxi_task_name.unique():
    print(j)
    t = LOT[(LOT.fuxi_task_name==j)]
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    for p in t.project_name.unique():
        axes[0].scatter(t[t.project_name==p].fuxi_max_cpu.values / t[t.project_name==p].fuxi_avg_cpu.values, t[t.project_name==p].fuxi_max_mem.values / t[t.project_name==p].fuxi_avg_mem.values, label=p)
    axes[0].legend()
    axes[0].set_xlabel("CPU Max/Avg")
    axes[0].set_ylabel("MEM Max/Avg")
#     for p in t.project_name.unique():
#         axes[1].scatter(t[t.project_name==p].fuxi_avg_mem.values, t[t.project_name==p].total_time_seconds.values, label=p)
#     axes[1].legend()
    plt.show()

In [ ]:
importance = [0.03497309, 0.03801977, 0.02930458, 0.01084006, 0.02969077,
       0.01236241, 0.0305444 , 0.02590316, 0.00272693, 0.00155067,
       0.00561709, 0.01129502, 0.00589236, 0.        , 0.        ,
       0.00291892, 0.00586741, 0.        , 0.00022326, 0.        ,
       0.05389414, 0.02888373, 0.04488752, 0.04728596, 0.04904115,
       0.02382207, 0.0377777 , 0.06192491, 0.04339618, 0.00538922,
       0.01949288, 0.04005669, 0.01613647, 0.00306931, 0.00173181,
       0.01232831, 0.02898802, 0.00559515, 0.00626779, 0.0349475 ,
       0.00037615, 0.00012161, 0.        , 0.00761775, 0.00068263,
       0.        , 0.        , 0.00551716, 0.00257583, 0.0031638 ,
       0.        , 0.00103412, 0.00766935, 0.00299292, 0.00491839,
       0.03316393, 0.0348981 , 0.01727366, 0.01821946, 0.04712876]
for cpu in [0, 1, 2]:
    for mem in [0, 1, 2, 3, 4]:
        for time in [0, 1, 2, 3]:
            index = 0
            if cpu == 0:
                idnex = 0
            elif cpu == 1:
                index = 1
            else:
                index = 2

            if mem == 0:
                index = (index*5) + 0
            elif mem == 1:
                index = (index*5) + 1
            elif mem == 2:
                index = (index*5) + 2
            elif mem == 3:
                index = (index*5) + 3
            else:
                index = (index*5) + 4

            if time == 0:
                index = (index*4) + 0
            elif time == 1:
                index = (index*4) + 1
            elif time == 2:
                index = (index*4) + 2
            else:
                index = (index*4) + 3
            print(cpu, mem, time ,index, importance[index])

In [ ]:
importance = [72468, 62357, 4, 253, 18991, 106108, 4, 16, 16483, 79888, 35, 7, 3702, 7133, 7, 1016, 2892, 28991, 21, 2255, 1680, 13002, 48, 172, 5580, 34147, 104, 172, 2297, 22334, 83, 2074, 710, 3647, 0, 69, 38, 230, 10, 10, 347, 1782, 20, 202, 15, 10508, 2, 607, 0, 919, 0, 1147, 0, 0, 0, 0, 0, 447, 0, 2, 0, 1527, 0, 37, 286, 2626, 0, 93, 5, 28, 0, 0, 0, 2, 0, 0, 0, 10, 0, 0, 248592, 920, 684, 0, 69490, 240, 507, 12, 85860, 1075, 431, 1, 186096, 0, 4484, 0, 104692, 933, 2621, 0, 33894, 300, 1387, 0, 82802, 4451, 2582, 3, 203757, 19273, 5649, 1246, 50170, 14922, 839, 3708, 2085, 63, 582, 16, 10628, 766, 469, 128, 86656, 16044, 2721, 3726, 22406, 3762, 806, 3853, 407, 14, 143, 74, 2098, 31, 94, 6, 8854, 3599, 270, 19, 45158, 9306, 1625, 2989, 2301, 520, 48, 47, 600, 839, 20, 5, 59742, 4746, 4480, 302, 6, 2212, 0, 0, 98, 300, 0, 0, 0, 654, 0, 0, 0, 3059, 0, 0, 879, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1870, 98, 0, 0, 581, 2, 21, 1, 528, 0, 0, 0, 113, 96, 0, 0, 229, 0, 0, 0, 6268, 4069, 605, 157, 2996, 31, 13, 13, 3129, 19, 15, 0, 16845, 52, 97, 0, 67712, 17742, 2246, 994, 18411, 1073, 267, 64, 14692, 641, 338, 17, 145830, 5173, 7924, 523]
for cpu in [0, 1, 2]:
    for mem in [0, 1, 2, 3, 4]:
        for time in [0, 1, 2, 3]:
            for stable in [0, 1, 2, 3]:
                index = 0
                if cpu == 0:
                    idnex = 0
                elif cpu == 1:
                    index = 1
                else:
                    index = 2

                if mem == 0:
                    index = (index*5) + 0
                elif mem == 1:
                    index = (index*5) + 1
                elif mem == 2:
                    index = (index*5) + 2
                elif mem == 3:
                    index = (index*5) + 3
                else:
                    index = (index*5) + 4

                if time == 0:
                    index = (index*4) + 0
                elif time == 1:
                    index = (index*4) + 1
                elif time == 2:
                    index = (index*4) + 2
                else:
                    index = (index*4) + 3
                    
                if stable == 0:
                    index = (index*4) + 0
                elif stable == 1:
                    index = (index*4) + 1
                elif stable == 2:
                    index = (index*4) + 2
                else:
                    index = (index*4) + 3
                print(cpu, mem, time, stable, index, importance[index])

In [ ]:
batch = pd.DataFrame()
for i in range(7):
    temp = pd.read_csv("/data/zkp/ump2/detailhost/d" + str(i) + ".csv")
    temp.dropna(inplace=True)
    batch = pd.concat([batch, temp])

In [ ]:
# SQL = G[G.index.str.contains("SQL")]
# SQLRT = G[G.index.str.contains("SQLRT")]
# Dup = G[G.index.str.contains("Dup")]
# COPY = G[G.index.str.contains("COPY")]
# Algo = G[G.index.str.contains("Algo")]
# LOT = G[G.index.str.contains("LOT")]
CUPID = G[G.index.str.contains("CUPID")]
# Merge = G[G.index.str.contains("Merge")]
# PS = G[G.index.str.contains("PS")]
# XLib = G[G.index.str.contains("XLib")]


In [ ]:
plt.figure(figsize=(8, 7), dpi=120)
for i, n in zip([PS, SQLRT, Dup, Algo], ["SQL", "SQLRT", "Dup", "Algo"]):
    plt.plot(sorted(i.fuxi_avg_cpu.values), np.arange(i.shape[0]) / (i.shape[0] - 1), label=n, lw=3)
plt.xlim(0, 200)
plt.legend()

In [ ]:
plt.figure(figsize=(8, 7), dpi=120)
for i, n in zip([PS, SQLRT, Dup, Algo], ["SQL", "SQLRT", "Dup", "Algo"]):
    plt.plot(sorted(i.fuxi_avg_mem.values), np.arange(i.shape[0]) / (i.shape[0] - 1), label=n, lw=4)
plt.xlim(0, 10000)
plt.legend()

In [ ]:
plt.figure(figsize=(8, 7), dpi=120)
for i, n in zip([PS, SQLRT, Dup, Algo], ["SQL", "SQLRT", "Dup", "Algo"]):
    plt.plot(sorted(i.total_time.values), np.arange(i.shape[0]) / (i.shape[0] - 1), label=n, lw=3)
plt.xlim(0, 200)
plt.legend()

In [ ]:
temp = Algo[Algo.index.str.contains("MWorker")].total_time.values
plt.plot(sorted(temp), np.arange(temp.shape[0]) / (temp.shape[0] - 1), lw=3)
plt.xlim(0, 600)

In [ ]:
job = pd.read_csv("/data/zkp/recurrent/job.csv")

In [ ]:
job[(job.fuxi_task_num==1) & (job.odps_task_type=="SQL")]

In [ ]:
data = pd.DataFrame()
for i in range(1, 8):
    file = pd.read_csv("/data/zkp/recurrent/d" + str(i) + ".csv")
    data = pd.concat([data, file])

In [ ]:
temp = data[(data.fuxi_job_name=='aliyun_searchlog_20220208033020139g97v24ys_SQL_0_1_0_job_0') & (data.fuxi_task_name=="M1")]

In [ ]:
for i in data.job_task_name.values[::1000]:
    print(i)
    temp = data[data.job_task_name==i]
    plt.scatter(temp.fuxi_avg_cpu, temp.fuxi_avg_mem)
    plt.show()

In [ ]:
data.sort_values(by=["fuxi_job_name", "fuxi_task_name"], inplace=True)

In [ ]:
task_inst_num = data[["fuxi_job_name", "fuxi_task_name", "fuxi_avg_cpu"]].groupby(["fuxi_job_name", "fuxi_task_name"]).count()

In [ ]:
task_inst_num.rename(columns={"fuxi_avg_cpu": "task_inst_num"}, inplace=True)

In [ ]:
task_inst_num.reset_index(inplace=True)

In [ ]:
index = task_inst_num.index
mpp = {}
for i in index:
    j = i[0]
    t = i[1]
    if j not in mpp:
        mpp[j] = ""
    mpp[j] += t

In [ ]:
mpp

In [ ]:
seq = []
for i in data.fuxi_job_name.values:
    seq.append(mpp[i])
data["task_seq"] = seq

In [ ]:
data = data[~pd.isna(data.fuxi_avg_cpu) & (~pd.isna(data.fuxi_avg_mem)) & (data.total_time_seconds>=0)]

In [ ]:
data["start_time"] = data.start_time % 86400
data["end_time"] = data.end_time % 86400

In [ ]:
G = data.groupby(["task_seq", "fuxi_job_name", "fuxi_task_name"]).mean()


In [ ]:
cnt = {}
for i in tempG.index:
    if i[0] not in cnt:
        cnt[i[0]] = []
    cnt[i[0]].append(i[1])

In [ ]:
len(cnt)

In [ ]:
temp = []
for key in cnt:
#     temp.append(len(cnt[key]))
    if len(cnt[key]) == 1:
        temp.append(cnt[key][0])
temp

In [ ]:
# unique = data[data.fuxi_job_name.isin(temp)]
unique = unique.groupby(["fuxi_job_name", "fuxi_task_name"]).count()
unique_task = {}
for i in unique.index:
    if i[0] not in unique_task:
        unique_task[i[0]] = []
    unique_task[i[0]].append(i[1])

In [ ]:
unique_cnt = {}
for key in unique_task:
    unique_cnt[key] = len(unique_task[key])
df = pd.DataFrame.from_dict(unique_cnt, orient="index")
df["fuxi_job_name"] = df.index

In [ ]:
df = pd.merge(df, job[["fuxi_job_name", "fuxi_task_num"]], how="left", on="fuxi_job_name")
df.drop(columns=["fuxi_job_name"], inplace=True)

In [ ]:
bincount = np.bincount(abs(df.values[:, 0] - df.values[:, 1]))
bincount[0], np.sum(bincount)

In [ ]:
temp = []
cpu = G.fuxi_avg_cpu.values
for i in range(cpu.shape[0]):
    index = 0
    if cpu[i] < 50:
        index = 0
    elif 50 <= cpu[i] < 150:
        index = 1
    else:
        index = 2
    temp.append(index)
G["fuxi_avg_cpu"]  = temp

In [ ]:
temp = []
mem = G.fuxi_avg_mem.values
for i in range(mem.shape[0]):
    if mem[i] < 512:
        index = 0
    elif 512 <= mem[i] < 1536:
        index = 1
    elif 1536 <= mem[i] < 3072:
        index = 2
    elif 3072 <= mem[i] < 6144:
        index = 3
    else:
        index = 4
    temp.append(index)
G["fuxi_avg_mem"] = temp

In [ ]:
temp = []
t = G.total_time_seconds.values
for i in range(t.shape[0]):
    if t[i] < 20:
        index = 0
    elif 20 <= t[i] < 40:
        index = 1
    elif 40 <= t[i] < 100:
        index = 2
    else:
        index = 3
    temp.append(index)
G["total_time_seconds"] = temp

In [ ]:
G

In [ ]:
G.reset_index(inplace=True)

In [ ]:
merge = pd.merge(G, job, how="left", on=["fuxi_job_name"])

In [ ]:
merge = pd.merge(merge, task_inst_num, how="left", on=["fuxi_job_name", "fuxi_task_name"])

In [ ]:
merge.T

In [ ]:
y = merge[["fuxi_avg_cpu", "fuxi_avg_mem", "total_time_seconds"]].values

In [ ]:
sy = (y[:, 0] * 5 + y[:, 1]) * 4  + y[:, 2]

In [ ]:
en = ["odps_task_type", "running_cluster", "fuxi_quota_name", "running_mode", "fuxi_quota_id"] #注意task_seq, fuxi_task_name
for i in en:
    merge[i] = LabelEncoder().fit_transform(merge[i])

In [ ]:
merge.fuxi_avg_cpu.value_counts()

In [ ]:
X = merge[["start_time_x", "odps_task_type", "fuxi_job_sn", "running_cluster", "fuxi_quota_id", "fuxi_quota_name", "fuxi_task_num",
          "fuxi_inst_num", "running_mode", "start_time_y", "plan_cpu_cost", "plan_mem_cost", "task_inst_num"]].values

In [ ]:
X

随机划分样本，CPU， MEM， Duration 0.82， 0.63，0.91
前六天，后一天，CPU， MEM，Duration，0.76， 0.60， 0.92

task数目 586069
占据数量前十的DAG结构，他们对应的预测精度
任务数目最多的DAG结构，他们对应的预测精度

In [ ]:
model = RandomForestClassifier()
X_train, X_test, y_train, y_test = X[merge.start_time_y<1644595200], X[merge.start_time_y>=1644595200], y[merge.start_time_y<1644595200, 2], y[merge.start_time_y>=1644595200, 2]
model.fit(X_train, y_train)
model.score(X_test, y_test)

In [ ]:
DAG = merge.task_seq.unique()

In [ ]:
df_DAG = []
df_trainsize = []
df_testsize = [] 
df_taskname = []
df_tasknum = []
df_result = []
df_tasktype = []
df_insnum = []
df_taskinstnum = []

df_predict = []
df_true = []


for dag, i in zip(DAG, range(DAG.shape[0])):
    print(i, end=', ')
    me = merge[merge.task_seq==dag]
    for t in me.fuxi_task_name.unique():
        temp = me[me.fuxi_task_name==t]
        result = [-1, -1, -1]

        predict_v = []
        y = temp[["fuxi_avg_cpu", "fuxi_avg_mem", "total_time_seconds"]].values
        X = temp[["start_time_x", "odps_task_type", "fuxi_job_sn", "running_cluster", "fuxi_quota_id", "fuxi_quota_name", "fuxi_task_num",
              "fuxi_inst_num", "running_mode", "start_time_y", "plan_cpu_cost", "plan_mem_cost", "task_inst_num"]].values  
        X_train, X_test, y_train, y_test = X[temp.start_time_y<1644595200], X[temp.start_time_y>=1644595200], y[temp.start_time_y<1644595200], y[temp.start_time_y>=1644595200]
        
        if X_train.shape[0]<6 or not X_test.shape[0]:
            continue
        
        for i in range(3):
            model = RandomForestClassifier()
            model.fit(X_train, y_train[:, i])
            result[i] = model.score(X_test, y_test[:, i])
            predict_v.append(model.predict(X_test))
        predict_value = np.asarray(predict_v).T
        true_value = y_test
        for s in range(true_value.shape[0]):
            df_true.append(true_value[s])
            df_predict.append(predict_value[s])   

            
        for s in X_test:
            df_DAG.append(dag)
            df_taskname.append(t)
            df_tasknum.append(s[6])
            df_insnum.append(s[7])
            df_taskinstnum.append(s[12])
            df_tasktype.append(s[1])
            df_result.append(result)
            df_totalresult.append(totalresult)
            df_trainsize.append(X_train.shape[0])
            df_testsize.append(X_test.shape[0])


In [ ]:
df = pd.DataFrame()
for name, i in zip(["DAG", "taskname", "tasknum", "insnum", "taskinstnum", "tasktype", "trainsize", "testsize"], [df_DAG, df_taskname, df_tasknum, df_insnum, df_taskinstnum, df_tasktype, df_trainsize, df_testsize]):
    df[name] = i
for name, i in zip(["cpu_result", "mem_result", "duration_result"], range(3)):
    df[name] = np.asarray(df_result)[:, i]

In [ ]:
for name, i in zip(["cpu_true", "mem_true", "duration_true"], range(3)):
    df[name] = np.asarray(df_true)[:, i]
for name, i in zip(["cpu_predict", "mem_predict", "duration_predict"], range(3)):
    df[name] = np.asarray(df_predict)[:, i]

In [ ]:
temp = []
for i in range(df.shape[0]):
    if df.cpu_true.values[i] == df.cpu_predict.values[i] and df.mem_true.values[i]==df.mem_predict.values[i] and df.duration_true.values[i]==df.duration_predict.values[i]:
        temp.append(1)
    else:
        temp.append(0)
df["totaltrue"] = temp

In [ ]:
temp = df[df.mem_true!=df.mem_predict].cpu.values
plt.plot(sorted(temp), np.arange(temp.shape[0]) / (temp.shape[0] - 1))
plt.xlim(0, 200)

In [ ]:
l = []
for dag, i in zip(DAG, range(DAG.shape[0])):
    print(i, end=', ')
    me = merge[merge.task_seq==dag]
    for t in me.fuxi_task_name.unique():
        temp = me[me.fuxi_task_name==t]
        X = temp[["fuxi_avg_cpu", "fuxi_avg_mem", "total_time_seconds"]].values
        X_train, X_test = X[temp.start_time_y<1644595200], X[temp.start_time_y>=1644595200]
        
        if X_train.shape[0]<6 or not X_test.shape[0]:
            continue
        for x in X_test:
            l.append(x)

In [ ]:
df.T

In [ ]:
job1 = df.groupby(["DAG", "taskname"]).count().sort_values(by=["tasknum"], ascending=False)
job1.reset_index(inplace=True)

In [ ]:
acc = []
for dag in job1.DAG.unique()[:10]:
    temp = df[df.DAG==dag]
    for task in temp.taskname.unique():
        tep = temp[temp.taskname==task]
        acc.append(np.sum(tep.totaltrue) / tep.shape[0])

In [ ]:
print(acc, sep=',')

In [ ]:
job2 = df.sort_values(by=["tasknum"], ascending=False)

In [ ]:
acc = []
for dag in job2.DAG.unique()[:10]:
    temp = df[df.DAG==dag]
    for task in temp.taskname.unique():
        tep = temp[temp.taskname==task]
        print(task, tep.tasknum.values[0], tep.shape[0], np.sum(tep.taskinstnum), np.sum(tep.totaltrue))

In [ ]:
job3 = df.sort_values(by=["taskinstnum"], ascending=False)

In [ ]:
job3.head(1000)

In [ ]:
job4 = df.groupby(["DAG", "taskname"]).mean().sort_values(by=["totaltrue"], ascending=False)

In [ ]:
job4[(job4.totaltrue>0.9) & (job4.testsize>10)].sort_values(by=["testsize"])

0: Algo
1: LOT
2: Merge
3: PS

In [ ]:
temp = df[(df.tasktype!=4) & (df.tasktype!=1)]
(temp.totaltrue * temp.taskinstnum).sum() / temp.taskinstnum.sum()


In [ ]:
df[df.tasktype==2]

In [ ]:
df[df.taskname=="Predict"].sort_values(by=["cpu_true", "mem_true", "duration_true"])

In [ ]:
merge[merge.fuxi_task_name=="Predict"].sort_values(by=["total_time_seconds"]).tail(20)

In [ ]:
for i in range(5):
    temp = df[df.tasktype==i]
    print(temp.totaltrue.sum() / temp.shape[0], end=', ')

In [ ]:
temp = []
cpu = data.fuxi_avg_cpu.values
for i in range(cpu.shape[0]):
    index = 0
    if cpu[i] < 50:
        index = 0
    elif 50 <= cpu[i] < 150:
        index = 1
    else:
        index = 2
    temp.append(index)
data["fuxi_avg_cpu_index"]  = temp
temp = []
mem = data.fuxi_avg_mem.values
for i in range(mem.shape[0]):
    if mem[i] < 512:
        index = 0
    elif 512 <= mem[i] < 1536:
        index = 1
    elif 1536 <= mem[i] < 3072:
        index = 2
    elif 3072 <= mem[i] < 6144:
        index = 3
    else:
        index = 4
    temp.append(index)
data["fuxi_avg_mem_index"] = temp
temp = []
t = data.total_time_seconds.values
for i in range(t.shape[0]):
    if t[i] < 20:
        index = 0
    elif 20 <= t[i] < 40:
        index = 1
    elif 40 <= t[i] < 100:
        index = 2
    else:
        index = 3
    temp.append(index)
data["total_time_seconds_index"] = temp

In [ ]:
data["category"] = data.fuxi_avg_cpu_index.astype(str) + data.fuxi_avg_mem_index.astype(str) + data.total_time_seconds_index.astype(str)


In [ ]:
np.savetxt("/data/zkp/BEList/MEM.txt", data.iloc[15000000:16000000].fuxi_avg_mem_index.values, fmt="%d")


In [ ]:
data.iloc[15000000:16000000].category.value_counts().shape


In [ ]:
import random
import os
import time


pods = ["spark-2dc6t", "spark-4dtmv", "spark-4mltm", "spark-5xwvb", "spark-6rl6h", "spark-72c2z", "spark-7whm8", "spark-bqtnn", "spark-d5vk8", "spark-g577r", "spark-lnzgq", "spark-zcw6h", "spark-zpvsw"]

# for pod in pods:
#     os.system("nohup kubectl exec " + pod + " ./spark-bench_2.3.0_0.4.0-RELEASE/bin/spark-bench.sh ./spark-bench_2.3.0_0.4.0-RELEASE/examples/data_kmeans0.conf > /dev/null 2>&1 &")
#     os.system("nohup kubectl exec " + pod + " ./spark-bench_2.3.0_0.4.0-RELEASE/bin/spark-bench.sh ./spark-bench_2.3.0_0.4.0-RELEASE/examples/data_kmeans1.conf > /dev/null 2>&1 &")
#     os.system("nohup kubectl exec " + pod + " ./spark-bench_2.3.0_0.4.0-RELEASE/bin/spark-bench.sh ./spark-bench_2.3.0_0.4.0-RELEASE/examples/data_sql.conf > /dev/null 2>&1 &")
# time.sleep(180)
# print("generate data finished")

cpu = np.loadtxt("./CPU.txt", dtype=int)
mem = np.loadtxt("./MEM.txt", dtype=int)
makespan = np.loadtxt("./Makespan.txt", dtype=int)

while True:
    for i in range(len(cpu)):
        job = "BE_{}_{}_{}.conf".format(cpu[i], mem[i], makespan[i])
        try:
            for pod in pods:
                os.system("nohup kubectl exec " + pod +  " ./spark-bench_2.3.0_0.4.0-RELEASE/bin/spark-bench.sh ./spark-bench_2.3.0_0.4.0-RELEASE/examples/" + job + " > /dev/null 2>&1 &")
            print("{}_{}_{}".format(cpu, mem, makespan))
        except:
            print("Error_{}_{}_{}".format(cpu, mem, makespan))
        time.sleep(5)


In [ ]:
cpu = np.loadtxt("/data/zkp/BEList/CPU.txt", dtype=int)
mem = np.loadtxt("/data/zkp/BEList/MEM.txt", dtype=int)
makespan = np.loadtxt("/data/zkp/BEList/Makespan.txt", dtype=int)

In [ ]:
import subprocess

In [ ]:
ov = np.loadtxt("/data/zkp/BEList/overview_cpu.txt")

In [ ]:
temp = []
for i in ov:
    t = 0
    if i < 40:
        t = 5
    elif i < 50:
        t = 3
    elif i < 60:
        t = 2
    else:
        t = 1.5
    temp.append(t)

In [ ]:
for i in temp:
    print(i, end=',')

In [ ]:
temp[19+24*3]